In [ ]:
import lib._util.visualplot as vp
import lib._util.mlpipe as mlpipe

# Pre-processing
from lib._class.DFDuplicateRemoval import DFDuplicateRemoval

# Feature scaling
from lib._class.DFStandardScaler import DFStandardScaler
from lib._class.DFMinMaxScaler import DFMinMaxScaler

# Clustering
from lib._class.DFKMeans import DFKMeans
from lib._class.DFGaussianMixture import DFGaussianMixture

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 100)

import numpy as np

# Scikit-Learn
from sklearn.model_selection import cross_val_score, StratifiedKFold, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.utils.class_weight import compute_class_weight

# # Imbalanced-Learn
from imblearn.pipeline import Pipeline
from imblearn.combine import SMOTEENN, SMOTETomek

# Plotly
import plotly.express as px

# Constant Variable

In [ ]:
SOURCE_PATH_DATA = 'resources/data/'
OUT_PATH_GRAPH   = 'resources/output/graph/'

# Phase 1 - Data Loading
- Reference: https://www.kaggle.com/mlg-ulb/creditcardfraud/home
- Time: Number of seconds elapsed between this transaction and the first transaction in the dataset
- V1-V28: May be result of a PCA dimensionality reduction to protect user identities and sensitive features
- Amount: Transaction amount
- Class: 1 for fraudulent transactions, 0 otherwise

In [ ]:
df_chunks = pd.read_csv(f'{SOURCE_PATH_DATA}creditcard.csv', sep=',', chunksize=50_000)
data_df   = pd.concat(df_chunks)

data_df.shape

In [ ]:
data_df.head()

In [ ]:
vp.faststat(data_df)

In [ ]:
vp.value_count(data_df, 'Class')

###### Histogram

In [ ]:
vp.histogram(data_df,
             bin_algo='count',
             max_col=4,
             title='Phase 1 - Histogram',
             out_path=OUT_PATH_GRAPH,
             layout_kwargs={'height': 2048})

###### Box

In [ ]:
vp.box(data_df,
       color='Class',
       max_col=4,
       title='Phase 1 - Box',
       out_path=OUT_PATH_GRAPH,
       layout_kwargs={
           'height': 2048,
           'legend_orientation': 'h'
       })

###### KDE

In [ ]:
vp.kde(data_df,
       color='Class',
       max_col=4,
       title='Phase 1 - KDE',
       out_path=OUT_PATH_GRAPH,
       layout_kwargs={
           'height': 2048,
           'legend_orientation': 'h'
       })

# Phase 2 - Data Preparation
- Remove duplicated data

In [ ]:
duplicate_removal = DFDuplicateRemoval(target='Class', keep='mean')
duplicate_removal.fit(data_df)

# Observe duplicated data
duplicate_df = duplicate_removal.duplicate_df

duplicate_df

In [ ]:
vp.value_count(duplicate_df, 'Class')

In [ ]:
# Observe if duplicated data are having different target label
vp.value_count(
    duplicate_df.groupby(duplicate_removal.subset).agg(
        Class=('Class', 'mean')
    ).reset_index(),
    'Class'
)

In [ ]:
# Remove duplicated data
data_df = duplicate_removal.transform(data_df)

data_df.shape

In [ ]:
vp.value_count(data_df, 'Class')

# Phase 3 - Classification
- Baseline

In [ ]:
metric_dict = {
    'weight_precision': [],
    'weight_recall':    [],
    'weight_f1':        [],
    'roc_auc':          [],
    'cv_score':         [],
    'method':           [],
}

In [ ]:
# Separate features & target
X, y = mlpipe.xy_split(data_df, 'Class')

vp.value_count(y.to_frame(), 'Class')
X.shape

In [ ]:
# Separate dataset
X_train, X_test, y_train, y_test = mlpipe.dataset_split(X, y, test_size=.3, stratify=y, random_state=0)

print('Train dataset:\n-----------------------')
vp.value_count(y_train.to_frame(), 'Class')
print('\nTest dataset:\n----------------------')
vp.value_count(y_test.to_frame(), 'Class')

In [ ]:
# Cross validation score
standard_scaler = DFStandardScaler(columns=['Time', 'Amount'])
minmax_scaler   = DFMinMaxScaler()

steps = [
    ('standard_scaler', standard_scaler),
    ('minmax_scaler', minmax_scaler),
]
pipeline = Pipeline(steps, verbose=True)
pipeline.fit(X_train)

model  = LogisticRegression(random_state=0, n_jobs=-1)
scores = cross_val_score(
    model,
    pipeline.transform(X_train),
    y_train,
    scoring='roc_auc',
    cv=StratifiedKFold(10),
    verbose=10,
    n_jobs=-1
)

print(f'Mean score: {np.mean(scores)}')

In [ ]:
model.fit(
    pipeline.transform(X_train),
    y_train
)

# Evaluate train data
mlpipe.eval_classif(
    y_train,
    model.predict(
        pipeline.transform(X_train)
    ),
    y_prob=model.predict_proba(
        pipeline.transform(X_train)
    )[:,-1]
)

In [ ]:
# Evaluate test data
eval_dict = mlpipe.eval_classif(
    y_test,
    model.predict(
        pipeline.transform(X_test)
    ),
    y_prob=model.predict_proba(
        pipeline.transform(X_test)
    )[:,-1],
    return_evaluation=True
)

metric_dict['method'].append('Baseline')
metric_dict['cv_score'].append(scores)
metric_dict['roc_auc'].append(eval_dict['roc_auc'])
metric_dict['weight_precision'].append(eval_dict['report']['weighted avg']['precision'])
metric_dict['weight_recall'].append(eval_dict['report']['weighted avg']['recall'])
metric_dict['weight_f1'].append(eval_dict['report']['weighted avg']['f1-score'])

# Phase 4 - Classification
- Class Weighting

### Class Weight

In [ ]:
classes           = np.unique(y_train)
weights           = compute_class_weight('balanced', classes, y_train)
class_weight_dict = {classes[i]: x for i,x in enumerate(weights)}

class_weight_dict

In [ ]:
# Cross validation score
steps = [
    ('standard_scaler', standard_scaler),
    ('minmax_scaler', minmax_scaler),
]
pipeline = Pipeline(steps, verbose=True)
pipeline.fit(X_train)

model  = LogisticRegression(class_weight=class_weight_dict, random_state=0, n_jobs=-1)
scores = cross_val_score(
    model,
    pipeline.transform(X_train),
    y_train,
    scoring='roc_auc',
    cv=StratifiedKFold(10),
    verbose=10,
    n_jobs=-1
)

print(f'Mean score: {np.mean(scores)}')

In [ ]:
model.fit(
    pipeline.transform(X_train),
    y_train
)

# Evaluate train data
mlpipe.eval_classif(
    y_train,
    model.predict(
        pipeline.transform(X_train)
    ),
    y_prob=model.predict_proba(
        pipeline.transform(X_train)
    )[:,-1]
)

In [ ]:
# Evaluate test data
eval_dict = mlpipe.eval_classif(
    y_test,
    model.predict(
        pipeline.transform(X_test)
    ),
    y_prob=model.predict_proba(
        pipeline.transform(X_test)
    )[:,-1],
    return_evaluation=True
)

metric_dict['method'].append('Class Weight')
metric_dict['cv_score'].append(scores)
metric_dict['roc_auc'].append(eval_dict['roc_auc'])
metric_dict['weight_precision'].append(eval_dict['report']['weighted avg']['precision'])
metric_dict['weight_recall'].append(eval_dict['report']['weighted avg']['recall'])
metric_dict['weight_f1'].append(eval_dict['report']['weighted avg']['f1-score'])

### Class Ratio

In [ ]:
# Reference:
# - https://machinelearningmastery.com/cost-sensitive-neural-network-for-imbalanced-classification/?fbclid=IwAR1PcEicqDXadG9hsNE-Tf4RQQ_DpIaCV4LRcuizGbTC9Ek5PiMbB_x26bU
# - https://www.youtube.com/watch?v=D6AChZlN5m0
n_class0          = y_train.value_counts().loc[0]
n_class1          = y_train.value_counts().loc[1]
class_weight_dict = {0: 1, 1: int(round(n_class0 / n_class1))}

class_weight_dict

In [ ]:
# Cross validation score
steps = [
    ('standard_scaler', standard_scaler),
    ('minmax_scaler', minmax_scaler),
]
pipeline = Pipeline(steps, verbose=True)
pipeline.fit(X_train)

model  = LogisticRegression(class_weight=class_weight_dict, random_state=0, n_jobs=-1)
scores = cross_val_score(
    model,
    pipeline.transform(X_train),
    y_train,
    scoring='roc_auc',
    cv=StratifiedKFold(10),
    verbose=10,
    n_jobs=-1
)

print(f'Mean score: {np.mean(scores)}')

In [ ]:
model.fit(
    pipeline.transform(X_train),
    y_train
)

# Evaluate train data
mlpipe.eval_classif(
    y_train,
    model.predict(
        pipeline.transform(X_train)
    ),
    y_prob=model.predict_proba(
        pipeline.transform(X_train)
    )[:,-1]
)

In [ ]:
# Evaluate test data
eval_dict = mlpipe.eval_classif(
    y_test,
    model.predict(
        pipeline.transform(X_test)
    ),
    y_prob=model.predict_proba(
        pipeline.transform(X_test)
    )[:,-1],
    return_evaluation=True
)

metric_dict['method'].append('Class Ratio')
metric_dict['cv_score'].append(scores)
metric_dict['roc_auc'].append(eval_dict['roc_auc'])
metric_dict['weight_precision'].append(eval_dict['report']['weighted avg']['precision'])
metric_dict['weight_recall'].append(eval_dict['report']['weighted avg']['recall'])
metric_dict['weight_f1'].append(eval_dict['report']['weighted avg']['f1-score'])

# Phase 5 - Classification
- Re-sampling

### SMOTE + ENN

In [ ]:
steps = [
    ('standard_scaler', standard_scaler),
    ('smote_enn', SMOTEENN(random_state=0, n_jobs=-1)),
]
X_bal, y_bal = Pipeline(steps, verbose=True).fit_resample(X_train, y_train)

vp.value_count(y_bal.to_frame(), 'Class')

In [ ]:
# Cross validation score
steps = [
    ('minmax_scaler', minmax_scaler),
]
pipeline = Pipeline(steps, verbose=True)
pipeline.fit(X_bal)

model  = LogisticRegression(random_state=0, n_jobs=-1)
scores = cross_val_score(
    model,
    pipeline.transform(X_bal),
    y_bal,
    scoring='roc_auc',
    cv=StratifiedKFold(10),
    verbose=10,
    n_jobs=-1
)

print(f'Mean score: {np.mean(scores)}')

In [ ]:
model.fit(
    pipeline.transform(X_bal),
    y_bal
)

# Evaluate train data (re-sample)
mlpipe.eval_classif(
    y_bal,
    model.predict(
        pipeline.transform(X_bal)
    ),
    y_prob=model.predict_proba(
        pipeline.transform(X_bal)
    )[:,-1]
)

In [ ]:
# Evaluate train data (original)
steps = [
    ('standard_scaler', standard_scaler),
    ('minmax_scaler', minmax_scaler),
]

mlpipe.eval_classif(
    y_train,
    model.predict(
        Pipeline(steps, verbose=True).transform(X_train)
    ),
    y_prob=model.predict_proba(
        Pipeline(steps, verbose=True).transform(X_train)
    )[:,-1]
)

In [ ]:
# Evaluate test data
steps = [
    ('standard_scaler', standard_scaler),
    ('minmax_scaler', minmax_scaler),
]

eval_dict = mlpipe.eval_classif(
    y_test,
    model.predict(
        Pipeline(steps, verbose=True).transform(X_test)
    ),
    y_prob=model.predict_proba(
        Pipeline(steps, verbose=True).transform(X_test)
    )[:,-1],
    return_evaluation=True
)

metric_dict['method'].append('SMOTE + ENN')
metric_dict['cv_score'].append(scores)
metric_dict['roc_auc'].append(eval_dict['roc_auc'])
metric_dict['weight_precision'].append(eval_dict['report']['weighted avg']['precision'])
metric_dict['weight_recall'].append(eval_dict['report']['weighted avg']['recall'])
metric_dict['weight_f1'].append(eval_dict['report']['weighted avg']['f1-score'])

### SMOTE + Tomek

In [ ]:
steps = [
    ('standard_scaler', standard_scaler),
    ('smote_tomek', SMOTETomek(random_state=0, n_jobs=-1)),
]
X_bal, y_bal = Pipeline(steps, verbose=True).fit_resample(X_train, y_train)

vp.value_count(y_bal.to_frame(), 'Class')

In [ ]:
# Cross validation score
steps = [
    ('minmax_scaler', minmax_scaler),
]
pipeline = Pipeline(steps, verbose=True)
pipeline.fit(X_bal)

model  = LogisticRegression(random_state=0, n_jobs=-1)
scores = cross_val_score(
    model,
    pipeline.transform(X_bal),
    y_bal,
    scoring='roc_auc',
    cv=StratifiedKFold(10),
    verbose=10,
    n_jobs=-1
)

print(f'Mean score: {np.mean(scores)}')

In [ ]:
model.fit(
    pipeline.transform(X_bal),
    y_bal
)

# Evaluate train data (re-sample)
mlpipe.eval_classif(
    y_bal,
    model.predict(
        pipeline.transform(X_bal)
    ),
    y_prob=model.predict_proba(
        pipeline.transform(X_bal)
    )[:,-1]
)

In [ ]:
# Evaluate train data (original)
steps = [
    ('standard_scaler', standard_scaler),
    ('minmax_scaler', minmax_scaler),
]

mlpipe.eval_classif(
    y_train,
    model.predict(
        Pipeline(steps, verbose=True).transform(X_train)
    ),
    y_prob=model.predict_proba(
        Pipeline(steps, verbose=True).transform(X_train)
    )[:,-1]
)

In [ ]:
# Evaluate test data
steps = [
    ('standard_scaler', standard_scaler),
    ('minmax_scaler', minmax_scaler),
]

eval_dict = mlpipe.eval_classif(
    y_test,
    model.predict(
        Pipeline(steps, verbose=True).transform(X_test)
    ),
    y_prob=model.predict_proba(
        Pipeline(steps, verbose=True).transform(X_test)
    )[:,-1],
    return_evaluation=True
)

metric_dict['method'].append('SMOTE + Tomek')
metric_dict['cv_score'].append(scores)
metric_dict['roc_auc'].append(eval_dict['roc_auc'])
metric_dict['weight_precision'].append(eval_dict['report']['weighted avg']['precision'])
metric_dict['weight_recall'].append(eval_dict['report']['weighted avg']['recall'])
metric_dict['weight_f1'].append(eval_dict['report']['weighted avg']['f1-score'])

# Phase 6 - Classification
- Clustering

### K-Means

In [ ]:
# Determine number of clusters
kmeans = DFKMeans(cluster_name='KMeans', n_clusters=15, random_state=0, n_jobs=-1,
                  eval_inertia=True, eval_silhouette=True, eval_chi=True, eval_dbi=True,
                  eval_sample_size=int(len(X_train) * .25))

steps = [
    ('standard_scaler', standard_scaler),
    ('kmeans', kmeans),
]
Pipeline(steps, verbose=True).fit(X_train)

###### Line

In [ ]:
vp.line(kmeans.eval_df,
        xy_tuples=[('n_cluster', x) for x in ['inertia', 'silhouette', 'calinski_harabasz', 'davies_bouldin']],
        max_col=2,
        title='Phase 6 - N Cluster - K-Means',
        out_path=OUT_PATH_GRAPH)

In [ ]:
# Determine number of clusters by scores
kmeans.eval_df.loc[kmeans.eval_df['silhouette'].idxmax()]['n_cluster'],\
kmeans.eval_df.loc[kmeans.eval_df['calinski_harabasz'].idxmax()]['n_cluster'],\
kmeans.eval_df.loc[kmeans.eval_df['davies_bouldin'].idxmin()]['n_cluster']

In [ ]:
# Clustering
kmeans = DFKMeans(cluster_name='KMeans', n_clusters=5, random_state=0, n_jobs=-1)

steps  = [
    ('standard_scaler', standard_scaler),
    ('kmeans', kmeans),
]
cluster_pipeline = Pipeline(steps, verbose=True).fit(X_train)

In [ ]:
classes           = np.unique(y_train)
weights           = compute_class_weight('balanced', classes, y_train)
class_weight_dict = {classes[i]: x for i,x in enumerate(weights)}

class_weight_dict

In [ ]:
# Cross validation score
steps = [
    ('minmax_scaler', minmax_scaler),
]
pipeline = Pipeline(steps, verbose=True)
pipeline.fit(
    cluster_pipeline.predict_proba(X_train)
)

model  = LogisticRegression(class_weight=class_weight_dict, random_state=0, n_jobs=-1)
scores = cross_val_score(
    model,
    pipeline.transform(
        cluster_pipeline.predict_proba(X_train)
    ),
    y_train,
    scoring='roc_auc',
    cv=StratifiedKFold(10),
    verbose=10,
    n_jobs=-1
)

print(f'Mean score: {np.mean(scores)}')

In [ ]:
model.fit(
    pipeline.transform(
        cluster_pipeline.predict_proba(X_train)
    ),
    y_train
)

# Evaluate train data
mlpipe.eval_classif(
    y_train,
    model.predict(
        pipeline.transform(
            cluster_pipeline.predict_proba(X_train)
        )
    ),
    y_prob=model.predict_proba(
        pipeline.transform(
            cluster_pipeline.predict_proba(X_train)
        )
    )[:,-1]
)

In [ ]:
# Evaluate test data
eval_dict = mlpipe.eval_classif(
    y_test,
    model.predict(
        pipeline.transform(
            cluster_pipeline.predict_proba(X_test)
        )
    ),
    y_prob=model.predict_proba(
        pipeline.transform(
            cluster_pipeline.predict_proba(X_test)
        )
    )[:,-1],
    return_evaluation=True
)

metric_dict['method'].append('K-Means')
metric_dict['cv_score'].append(scores)
metric_dict['roc_auc'].append(eval_dict['roc_auc'])
metric_dict['weight_precision'].append(eval_dict['report']['weighted avg']['precision'])
metric_dict['weight_recall'].append(eval_dict['report']['weighted avg']['recall'])
metric_dict['weight_f1'].append(eval_dict['report']['weighted avg']['f1-score'])

### Gaussian Mixture

In [ ]:
# Determine number of clusters
gmm = DFGaussianMixture(cluster_name='GMM', n_components=15, max_iter=1_000, random_state=0,
                        eval_aic=True, eval_bic=True, eval_silhouette=True, eval_chi=True, eval_dbi=True,
                        eval_sample_size=int(len(X_train) * .25))

steps = [
    ('standard_scaler', standard_scaler),
    ('gmm', gmm),
]
Pipeline(steps, verbose=True).fit(X_train)

###### Line

In [ ]:
vp.line(gmm.eval_df,
        xy_tuples=[('n_cluster', x) for x in ['akaike', 'bayesian', 'silhouette', 'calinski_harabasz', 'davies_bouldin']],
        max_col=3,
        title='Phase 6 - N Cluster - GMM',
        out_path=OUT_PATH_GRAPH)

In [ ]:
# Determine number of clusters by scores
gmm.eval_df.loc[gmm.eval_df['akaike'].idxmin()]['n_cluster'],\
gmm.eval_df.loc[gmm.eval_df['bayesian'].idxmin()]['n_cluster'],\
gmm.eval_df.loc[gmm.eval_df['silhouette'].idxmax()]['n_cluster'],\
gmm.eval_df.loc[gmm.eval_df['calinski_harabasz'].idxmax()]['n_cluster'],\
gmm.eval_df.loc[gmm.eval_df['davies_bouldin'].idxmin()]['n_cluster']

In [ ]:
# Clustering
gmm = DFGaussianMixture(cluster_name='GMM', n_components=3, max_iter=1_000, random_state=0)

steps  = [
    ('standard_scaler', standard_scaler),
    ('gmm', gmm),
]
cluster_pipeline = Pipeline(steps, verbose=True).fit(X_train)

In [ ]:
classes           = np.unique(y_train)
weights           = compute_class_weight('balanced', classes, y_train)
class_weight_dict = {classes[i]: x for i,x in enumerate(weights)}

class_weight_dict

In [ ]:
# Cross validation score
steps = [
    ('minmax_scaler', minmax_scaler),
]
pipeline = Pipeline(steps, verbose=True)
pipeline.fit(
    cluster_pipeline.predict_proba(X_train)
)

model  = LogisticRegression(class_weight=class_weight_dict, random_state=0, n_jobs=-1)
scores = cross_val_score(
    model,
    pipeline.transform(
        cluster_pipeline.predict_proba(X_train)
    ),
    y_train,
    scoring='roc_auc',
    cv=StratifiedKFold(10),
    verbose=10,
    n_jobs=-1
)

print(f'Mean score: {np.mean(scores)}')

In [ ]:
model.fit(
    pipeline.transform(
        cluster_pipeline.predict_proba(X_train)
    ),
    y_train
)

# Evaluate train data
mlpipe.eval_classif(
    y_train,
    model.predict(
        pipeline.transform(
            cluster_pipeline.predict_proba(X_train)
        )
    ),
    y_prob=model.predict_proba(
        pipeline.transform(
            cluster_pipeline.predict_proba(X_train)
        )
    )[:,-1]
)

In [ ]:
# Evaluate test data
eval_dict = mlpipe.eval_classif(
    y_test,
    model.predict(
        pipeline.transform(
            cluster_pipeline.predict_proba(X_test)
        )
    ),
    y_prob=model.predict_proba(
        pipeline.transform(
            cluster_pipeline.predict_proba(X_test)
        )
    )[:,-1],
    return_evaluation=True
)

metric_dict['method'].append('GMM')
metric_dict['cv_score'].append(scores)
metric_dict['roc_auc'].append(eval_dict['roc_auc'])
metric_dict['weight_precision'].append(eval_dict['report']['weighted avg']['precision'])
metric_dict['weight_recall'].append(eval_dict['report']['weighted avg']['recall'])
metric_dict['weight_f1'].append(eval_dict['report']['weighted avg']['f1-score'])

# Phase 7 - Classification
- Hyperparameter Tuning

In [ ]:
classes           = np.unique(y_train)
weights           = compute_class_weight('balanced', classes, y_train)
class_weight_dict = {classes[i]: x for i,x in enumerate(weights)}

class_weight_dict

In [ ]:
# Reference: https://towardsdatascience.com/logistic-regression-model-tuning-with-scikit-learn-part-1-425142e01af5
model  = LogisticRegression(class_weight=class_weight_dict, random_state=0, n_jobs=-1)
search = RandomizedSearchCV(
    estimator=model,
    param_distributions={
        'C': np.logspace(-4, 4, 20),
        'penalty': ['l1', 'l2'],
    },
    scoring='roc_auc',
    cv=StratifiedKFold(n_splits=10),
    n_jobs=-1,
    verbose=10,
    n_iter=100,
    random_state=0
)
search.fit(X_train, y_train)

In [ ]:
result_df = pd.DataFrame(search.cv_results_)
result_df[['params', 'mean_test_score', 'rank_test_score']].sort_values(by='mean_test_score', ascending=False)

In [ ]:
search.best_params_

In [ ]:
# Cross validation score
steps = [
    ('standard_scaler', standard_scaler),
    ('minmax_scaler', minmax_scaler),
]
pipeline = Pipeline(steps, verbose=True)
pipeline.fit(X_train)

model  = LogisticRegression(**search.best_params_, class_weight=class_weight_dict, random_state=0, n_jobs=-1)
scores = cross_val_score(
    model,
    pipeline.transform(X_train),
    y_train,
    scoring='roc_auc',
    cv=StratifiedKFold(10),
    verbose=10,
    n_jobs=-1
)

print(f'Mean score: {np.mean(scores)}')

In [ ]:
model.fit(
    pipeline.transform(X_train),
    y_train
)

# Evaluate train data
mlpipe.eval_classif(
    y_train,
    model.predict(
        pipeline.transform(X_train)
    ),
    y_prob=model.predict_proba(
        pipeline.transform(X_train)
    )[:,-1]
)

In [ ]:
# Evaluate test data
eval_dict = mlpipe.eval_classif(
    y_test,
    model.predict(
        pipeline.transform(X_test)
    ),
    y_prob=model.predict_proba(
        pipeline.transform(X_test)
    )[:,-1],
    return_evaluation=True
)

metric_dict['method'].append('CV Search')
metric_dict['cv_score'].append(scores)
metric_dict['roc_auc'].append(eval_dict['roc_auc'])
metric_dict['weight_precision'].append(eval_dict['report']['weighted avg']['precision'])
metric_dict['weight_recall'].append(eval_dict['report']['weighted avg']['recall'])
metric_dict['weight_f1'].append(eval_dict['report']['weighted avg']['f1-score'])

###### Bar

In [ ]:
eval_df  = pd.DataFrame(metric_dict)
eval_df['cv_mean'] = eval_df['cv_score'].apply(np.mean)

metrics  = [x for x in eval_df.columns if x not in ['cv_score', 'method']]
eval_dfs = [eval_df[[x, 'method']].rename(columns={x: 'score'}) for x in metrics]
for i,x in enumerate(eval_dfs):
    x['metric'] = metrics[i]

fig = px.bar(
    pd.concat(eval_dfs, axis=0),
    x='metric',
    y='score',
    color='method',
    barmode='group'
)
fig['layout']['legend_orientation'] = 'h'

vp.generate_plot(
    fig,
    out_path=OUT_PATH_GRAPH,
    out_filename='Phase 7 - Bar - Metrics Comparison')

# Phase 8 - Classification
- Stack Models

In [ ]:
# TODO